# Notebook para fozar


In [71]:
from conllu import parse

filename = "data/english/en_ewt-ud-dev.conllu"


def get_fields(line):
    words = line.split("\t")
    if len(words) < 4:
        return None
    return words[1], words[3], line


with open(filename, "r", encoding="utf-8") as file:
    sentences = parse(file.read())

sentences[:2]

[TokenList<From, the, AP, comes, this, story, :, metadata={newdoc id: "weblog-blogspot.com_nominations_20041117172713_ENG_20041117_172713", sent_id: "weblog-blogspot.com_nominations_20041117172713_ENG_20041117_172713-0001", newpar id: "weblog-blogspot.com_nominations_20041117172713_ENG_20041117_172713-p0001", text: "From the AP comes this story :"}>, TokenList<President, Bush, on, Tuesday, nominated, two, individuals, to, replace, retiring, jurists, on, federal, courts, in, the, Washington, area, ., metadata={sent_id: "weblog-blogspot.com_nominations_20041117172713_ENG_20041117_172713-0002", newpar id: "weblog-blogspot.com_nominations_20041117172713_ENG_20041117_172713-p0002", text: "President Bush on Tuesday nominated two individuals to replace retiring jurists on federal courts in the Washington area."}>]

In [76]:
X_data = []
y_data = []

for sentence in sentences:
    parsed_sentence = []
    sentence_labels = []
    for token in sentence:
        if type(token["id"]) == int:
            parsed_sentence.append(token["form"])
            sentence_labels.append(token["upostag"])
    X_data.append(" ".join(parsed_sentence))
    y_data.append(sentence_labels)


print(f"Sentences: {X_data[:5]}")
print(f"Class labels: {y_data[:5]}")

Sentences: ['From the AP comes this story :', 'President Bush on Tuesday nominated two individuals to replace retiring jurists on federal courts in the Washington area .', 'Bush nominated Jennifer M. Anderson for a 15 - year term as associate judge of the Superior Court of the District of Columbia , replacing Steffen W. Graae .', '***', 'Bush also nominated A. Noel Anketell Kramer for a 15 - year term as associate judge of the District of Columbia Court of Appeals , replacing John Montague Steadman .']
Class labels: [['ADP', 'DET', 'PROPN', 'VERB', 'DET', 'NOUN', 'PUNCT'], ['PROPN', 'PROPN', 'ADP', 'PROPN', 'VERB', 'NUM', 'NOUN', 'PART', 'VERB', 'VERB', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'ADP', 'DET', 'PROPN', 'NOUN', 'PUNCT'], ['PROPN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'ADP', 'DET', 'NUM', 'PUNCT', 'NOUN', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'PROPN', 'ADP', 'DET', 'PROPN', 'ADP', 'PROPN', 'PUNCT', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT'], ['PUNCT'], ['PROPN', 'ADV', 'VE